**DATACHALLENGE BDGIA DEBIASING MODEL**
---

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from evaluator import *

In [2]:

##############################################################
#  DEFINE CUSTOM LOSS FUNCTION AND EVALUATION FUNCTIONS
#   
#   soft_f1_loss
#   macro_soft_f1_loss
#   calculate_exact_macro_f1
#   calculate_class_tpr_gap
#   average_tpr_gap_per_class
#   
##############################################################


def soft_macro_f1_loss(y_true, y_pred):
    """
    Differentiable approximation of the macro F1 score as a loss function.
    Calculates the F1 score for each class independently and then takes the average.
    Inputs :
        y_true must be one hot encoded
    """
    y_pred_one_hot = torch.nn.functional.one_hot(y_pred, num_classes=Y_train.nunique()) if len(y_pred.shape) == 1 else y_pred
    #y_pred_probs = torch.softmax(y_pred_one_hot, dim=1)
    
    tp = torch.sum(y_true * y_pred, dim=0)
    pp = torch.sum(y_pred, dim=0)
    ap = torch.sum(y_true, dim=0)
    
    precision = tp / (pp + 1e-6)
    recall = tp / (ap + 1e-6)
    
    f1_per_class = 2 * (precision * recall) / (precision + recall + 1e-6)
    macro_f1 = torch.mean(f1_per_class)   # Mean to aggregate over all classes
    
    loss = 1 - macro_f1  # Minimizing loss is maximizing macro F1 score
    return loss


def get_macro_f1(y_true, y_pred):
    """
    Calculate the exact macro F1 score for evaluation.
    Input : any format as tensors will be converted to Tensors of true label if dim >1 . Can be :
        - Tensor of probabilities(y_pred_probs) dimension (n,28)
        - Tensor of labels, one hote encoded (y_pred_one_hot) dimension (n,28)
        - Tensor of labels (y_pred_tensor) dimension (n,1)
    Ouput : scalar
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true

    " predict macro f1"
    f1 = f1_score(y_true_labels.cpu().numpy(), y_pred_labels.cpu().numpy(), average='macro')
    return f1

def get_tpr_gap(y_true, y_pred, protected_attribute, class_idx):
    """
    Calculate the TPR gap for a specific class across protected groups.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred_probs: Tensor of predicted probabilities (after softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    - class_idx: Index of the class for which to calculate the TPR gap.
    
    Returns:
    - TPR gap for the specified class.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true
    
    # Calculate overall TPR for the current class
    overall_mask = y_true_labels == class_idx
    overall_tpr = torch.sum((y_pred_labels == class_idx) & overall_mask).float() / (torch.sum(overall_mask).float() + 1e-6)
    
    # Initialize list to store TPR for each protected group
    group_tprs = []
    
    # Calculate TPR for each protected group
    for group_val in protected_attribute.unique():
        group_mask = (protected_attribute == group_val) & overall_mask
        group_tpr = torch.sum((y_pred_labels == class_idx) & group_mask).float() / (torch.sum(group_mask).float() + 1e-6)
        group_tprs.append(group_tpr)
    
    # Calculate TPR gap for the current class
    tpr_gaps = torch.abs(torch.tensor(group_tprs) - overall_tpr)
    
    return torch.mean(tpr_gaps)  # Return the mean TPR gap for this class

def get_macro_tpr_gap(y_true, y_pred, protected_attribute):
    """
    Calculate the average TPR gap per class by calling tpr_gap for each class.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred: Tensor of predicted logits (before softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    
    Returns:
    - Average TPR gap across all classes.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true
    
    # Initialize list to store TPR gaps for all classes
    class_tpr_gaps = []
    
    # Iterate over each class
    num_classes = len(y_true_labels.unique())
    for class_idx in range(num_classes):
        class_tpr_gap = get_tpr_gap(y_true_labels, y_pred_labels, protected_attribute, class_idx)
        class_tpr_gaps.append(class_tpr_gap)
    
    # Calculate the average TPR gap across all classes
    avg_tpr_gap = torch.mean(torch.stack(class_tpr_gaps))
    
    return avg_tpr_gap


def soft_final_score_loss(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    soft_macro_f1 = soft_macro_f1_loss(y_true, y_pred)  # Calculate soft macro F1 score
    macro_tpr_gap = get_macro_tpr_gap(y_true, y_pred, protected_attribute)  # Calculate TPR gap
    
    soft_final_score = ( soft_macro_f1 + (1 - macro_tpr_gap) ) / 2
    return soft_final_score

def get_final_score(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true

    macro_f1 = get_macro_f1(y_true_labels, y_pred_labels)  # Calculate macro F1 score
    macro_tpr_gap = get_macro_tpr_gap(y_true_labels, y_pred_labels, protected_attribute)  # Calculate macro TPR gap
    
    final_score = (macro_f1 + (1 - macro_tpr_gap)) / 2
    return final_score




**LOAD AND PREPARE**
---

In [3]:
##############################################################
# LOAD DATA, 
#############################################################

# Load pickle file and convert to numpy array
with open('data-challenge-student.pickle', 'rb') as handle:
    # dat = pickle.load(handle)
    dat = pd.read_pickle(handle)
 
#Check keys()
print(dat.keys())
X = dat['X_train']
Y = dat['Y']
S = dat['S_train']

#create a label to distiguish 56 labels Y x 2 (man or woman)
# 0 to 27 = non sensitive group | 28 + [0 , 27] = 28 to 55 = sensitive group
Y56 = Y+28*S

X_test_true = dat['X_test']
S_test_true = dat['S_test']

# check size
print(X.shape,Y.shape,S.shape,X_test_true.shape,S_test_true.shape)


dict_keys(['X_train', 'X_test', 'Y', 'S_train', 'S_test'])
(27749, 768) (27749,) (27749,) (11893, 768) (11893,)


In [4]:
##############################################################
# train_test_split (np.arrays)
##############################################################

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y56_train, Y56_test = train_test_split(X, Y56, test_size=0.2, random_state=42)
Y_train = Y56_train % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train = Y56_train//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_test = Y56_test % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_test = Y56_test//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)

# impression des dimensions
print('train:',X_train.shape,Y_train.shape,S_train.shape)
print('test:',X_test.shape,Y_test.shape, S_test.shape)

##############################################################
# 1. Transform DataFrames into Tensors
##############################################################

X_tensor = torch.tensor(X.values, dtype=torch.float32)
Y_tensor = torch.tensor(Y.values, dtype=torch.long)
S_tensor = torch.tensor(S.values, dtype=torch.long)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.long)
S_train_tensor = torch.tensor(S_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.long)
S_test_tensor = torch.tensor(S_test.values, dtype=torch.long)

Y_train_one_hot = torch.nn.functional.one_hot(Y_train_tensor, num_classes=Y_train.nunique())
Y_test_one_hot = torch.nn.functional.one_hot(Y_test_tensor, num_classes=Y_train.nunique())

X_test_true_tensor = torch.tensor(X_test_true.values, dtype=torch.float32)

# impression des dimensions
print('train_tensor:',X_train_tensor.shape,Y_train_tensor.shape,S_train_tensor.shape, type(X_train_tensor))
print('test_tensor:',X_test_tensor.shape,Y_test_tensor.shape, S_test_tensor.shape, type(X_test_tensor))
print('Y_train_one_hot:',Y_train_one_hot.shape, type(Y_train_one_hot))
print('X_test_true_tensor:',X_test_true_tensor.shape, type(X_test_true_tensor))

train: (22199, 768) (22199,) (22199,)
test: (5550, 768) (5550,) (5550,)
train_tensor: torch.Size([22199, 768]) torch.Size([22199]) torch.Size([22199]) <class 'torch.Tensor'>
test_tensor: torch.Size([5550, 768]) torch.Size([5550]) torch.Size([5550]) <class 'torch.Tensor'>
Y_train_one_hot: torch.Size([22199, 28]) <class 'torch.Tensor'>
X_test_true_tensor: torch.Size([11893, 768]) <class 'torch.Tensor'>


**Evaluation functions**
---

In [ ]:
# FUNCTIONS

# to show performance

def evaluate(Y_pred,Y,S,will_print=1):
    '''returns model accuracy, final score, macro fscore ans TPR gap
    input : 2 np arrays of same dimension
    output : array of 4 values
    '''
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    print(f"Accuracy on transformed test data: {accuracy}")
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

    if will_print==1:
        #print results
        print('final score',final_score)
        print('macro_fscore',eval_scores['macro_fscore'])
        print('1-eval_scores[\'TPR_GAP\']',1-eval_scores['TPR_GAP'])
    
    return accuracy, final_score, eval_scores['macro_fscore'],1-eval_scores['TPR_GAP'] , eval_scores , confusion_matrices_eval

# to predict X_test and save to file

def save_Y_pred_tofile(X, model, name): # adapted to torch
    
    # save probabilities for each Xi (dim=28)
    y_pred_probs = model(X)
    probs=pd.DataFrame(y_pred_probs.detach().numpy(), columns= list(range(0,28)))
    file_name_probs = "y_pred_probs/y_pred_probs_"+str(name)+".csv"
    probs.to_csv(file_name_probs, header = None, index = None)

    # save predicted labels for each Xi (dim=1)
    y_pred = torch.argmax(y_pred_probs, dim=1)
    results=pd.DataFrame(y_pred.numpy(), columns= ['score'])
    file_name = "y_pred/Data_Challenge_"+str(name)+".csv"
    results.to_csv(file_name, header = None, index = None)

    return y_pred, y_pred_probs
    

def print_cassif_report(Y_pred,Y_test):
    # Convert Y_pred to a DataFrame
    Y_pred_df = pd.DataFrame(Y_pred_tensor.numpy(), columns=['Predicted'])

    # Evaluate Y_pred compared to Y_test (assuming Y_test is a numpy array or a pandas Series)
    table = classification_report(Y_test, Y_pred_df['Predicted'])

    return table

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        81
           1       0.00      0.00      0.00       127
           2       0.00      0.00      0.00       458
           3       0.00      0.00      0.00        36
           4       0.00      0.00      0.00        48
           5       0.00      0.00      0.00        72
           6       0.00      0.00      0.00       178
           7       0.00      0.00      0.00        54
           8       0.00      0.00      0.00        18
           9       0.00      0.00      0.00        91
          10       0.00      0.00      0.00        22
          11       0.00      0.00      0.00       286
          12       0.00      0.00      0.00       110
          13       0.00      0.00      0.00       258
          14       0.00      0.00      0.00       112
          15       0.00      0.00      0.00        19
          16       0.00      0.00      0.00        33
          17       0.00    

/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**ARRET ANTICIPE DU NN (sans mini-batch)**
---

In [29]:
#################################################
#          TEST DES PARAMETRES
################################################


# 1. Define the model and optimizer and train
# --------------------------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    #nn.LogSoftmax(dim=1)
    )  # LogSoftmax for multi-class classification

learning_rate=0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
num_epochs = 10000 

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
name = 'NN-28-28_Adam'+'_lr_'+str(learning_rate)
print('\n\n Starting to train model', name)
model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap = train_NN_with_custom_loss_no_mini_batch(model,optim.Adam(model.parameters(), lr=learning_rate), X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
#Res.loc[i]=[name,optimizer,learning_rate,final_score, macro_f1, inv_macro_tpr_gap]
#save_Y_pred_tofile(X_test_true_tensor, model_trained,name)




 Starting to train model NN-28-28_Adam_lr_0.01
Arrêt précoce après 12 époques
Final Evaluation Score: 0.6730861067771912 Macro F1: 0.4465380708164603 1-TPR_gap: 0.8996341228485107


**FUNCTION FOR NN WITH CUSTOM LOSS (INITIAL)**
---

In [27]:
# AVEC MINI BATCH

def train_NN_with_custom_loss(model, optimizer, batch_size, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor):

    # 1. Convertir les tensors en datasets puis en DataLoader pour gérer les mini-batchs
    train_dataset = TensorDataset(X_train_tensor, Y_train_one_hot, S_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = TensorDataset(X_test_tensor, Y_test_one_hot, S_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    
    # 2. Paramètres pour l'arrêt précoce
    # -------------------------------
    patience = 5  # Nombre d'époques à attendre après la dernière amélioration de la perte de validation
    best_loss = None
    early_ending = None
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        
        # 1/ exécuter les minibatches et recupérer la loss moyenne
        for X_batch, Y_batch, S_batch in train_loader:
            # Y_batch est one hot
            
            model.train()
            optimizer.zero_grad()
            outputs_train = model(X_batch)
            loss = soft_final_score_loss(Y_batch, outputs_train, S_batch)
            loss.backward()
            optimizer.step()

            # save mini-batch loss
            train_loss += loss.item()
        
        # Average loss pour l'epoch (après boucle mini-batchs)
        train_loss = train_loss / len(train_loader)       
        
        # 2. Vérifier si la perte de validation s'est améliorée (arret précoce)

        # Evaluation sur le jeu de données de test
        model.eval()
        test_loss = 0.0
        
        with torch.no_grad():
            for X_batch_test, Y_batch_test, S_batch_test in test_loader:
                outputs_test = model(X_batch_test)
                #Y_batch_test_one_hot = torch.nn.functional.one_hot(Y_batch_test, num_classes=Y_train.nunique())
                loss_test = soft_final_score_loss(Y_batch_test, outputs_test, S_batch_test)
                test_loss += loss_test.item()
                
        #average_test_loss = running_loss_test / len(test_loader)
        test_loss = test_loss / len(test_loader)
       
        # check if improvement in loss (compared to last epoch)
        if best_loss is None or test_loss < best_loss:
            best_loss = test_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f'Arrêt précoce après {epoch+1} époques')
                early_ending = epoch + 1
                break  # Arrêter l'entraînement
        
        # 3. Impression de l'apprentissage et des scores train et test
        if epoch==0 or (epoch+1) % 10 == 0:
            
            with torch.no_grad():
                
                # Calculate metrics for training data
                outputs_train = model(X_train_tensor) # probabilities
                # Evaluate predictions on training data
                final_score_train_ = get_final_score(Y_train_tensor, outputs_train, S_train_tensor)
                macro_f1_train = get_macro_f1(Y_train_tensor, outputs_train)
                inv_macro_tpr_gap_train = 1 - get_macro_tpr_gap(Y_train_tensor, outputs_train, S_train_tensor)
            
                # Calculate metrics for test data
                outputs_test = model(X_test_tensor)
                # Evaluate predictions on training data
                final_score_test_ = get_final_score(Y_test_tensor, outputs_test, S_test_tensor)
                macro_f1_test = get_macro_f1(Y_test_tensor, outputs_test)
                inv_macro_tpr_gap_test = 1 - get_macro_tpr_gap(Y_test_tensor, outputs_test, S_test_tensor)

                print(f'Epoch {epoch+1}, Loss: {loss.item()}, Final Score Train: {final_score_train_.item()}, Final Score Test: {final_score_test_.item()} (gap {final_score_test_-final_score_train_}) macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')
            
    # 4. Make Predictions and Evaluate with final_score
    # -------------------------------------------------
            
    with torch.no_grad():
        model.eval()

        Y_train_pred_probs = model(X_train_tensor) # dim = 28 (Probabilities for each class)
        
        # # Y_train_pred_tensor = torch.argmax(Y_train_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
        final_score_train = get_final_score(Y_train_tensor, Y_train_pred_probs, S_train_tensor)

        Y_pred_probs = model(X_test_tensor) # dim = 28 (Probabilities for each class)
        # Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
        macro_f1 = get_macro_f1(Y_test_tensor, Y_pred_tensor)
        inv_macro_tpr_gap = 1 - get_macro_tpr_gap(Y_test_tensor, Y_pred_probs, S_test_tensor)
        final_score = get_final_score(Y_test_tensor, Y_pred_probs, S_test_tensor)
        
        print(f'Final Evaluation Score: {final_score.item()} gap {final_score.item()-final_score_train.item()} || Macro F1: {macro_f1.item()} 1-TPR_gap: { inv_macro_tpr_gap.item() }')

    return model, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap, early_ending,final_score_train

In [28]:
#################################################
#          TEST DES PARAMETRES
################################################


# 1. Define the model and optimizer and train
# --------------------------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),
    nn.LogSoftmax(dim=1),  # LogSoftmax for multi-class classification
    )  

batch_size = 128
learning_rate=0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)#, weight_decay=0.001)
num_epochs = 1000

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
name = 'NN-28-28_Adam'+'_lr_'+str(learning_rate)+'_batch_size_'+str(batch_size)
print('\n\n Starting to train model', name)
model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap, early_ending, final_score_train = train_NN_with_custom_loss(model,optim.Adam(model.parameters(), lr=learning_rate), batch_size, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
#Res.loc[i]=[name,optimizer,learning_rate,batch_size, early_ending,final_score_train, final_score, macro_f1, inv_macro_tpr_gap]
save_Y_pred_tofile(X_test_true_tensor, model_trained,name)




 Starting to train model NN-28-28_Adam_lr_0.01_batch_size_128
Epoch 1, Loss: 0.9728055000305176, Final Score Train: 0.5082315802574158, Final Score Test: 0.5081309080123901 (gap -0.00010067224502563477) macro F1 Train: 0.016463136835433007, macro F1 Test: 0.016261778855711503, 1-TPR Gap Train: 1.0, 1-TPR Gap Test: 1.0
Epoch 10, Loss: 0.9646603465080261, Final Score Train: 0.5012662410736084, Final Score Test: 0.5011098384857178 (gap -0.000156402587890625) macro F1 Train: 0.0025325098538220418, macro F1 Test: 0.002219672785315243, 1-TPR Gap Train: 1.0, 1-TPR Gap Test: 1.0
Arrêt précoce après 19 époques
Final Evaluation Score: 0.5017502307891846 gap 4.07099723815918e-05 || Macro F1: 0.0162663060769965 1-TPR_gap: 1.0


(tensor([11, 11, 11,  ..., 11, 11, 11]),
 tensor([[-1019.4708, -1053.7491,  -901.7321,  ...,  -405.2040,  -953.3958,
          -1096.1006],
         [-1084.8196, -1147.7852, -1005.8746,  ...,  -570.9733, -1041.9598,
          -1120.8088],
         [ -808.4445,  -846.9246,  -729.4754,  ...,  -329.8860,  -764.5796,
           -864.8046],
         ...,
         [-1098.2998, -1161.5825, -1016.0308,  ...,  -549.1259, -1055.2952,
          -1133.0596],
         [ -954.2846,  -952.4446,  -838.9060,  ...,  -361.5235,  -856.6089,
           -929.9841],
         [-1117.0275, -1160.0225, -1040.0208,  ..., -1203.8389, -1157.7285,
          -1280.2397]], grad_fn=<LogSoftmaxBackward0>))

In [29]:
#################################################
#          BOUCLE HYPERPARAMETRES
################################################


# 1. Define the model and optimizer and train
# --------------------------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),  # Additional layer for complexity
    #nn.LogSoftmax(dim=1))  # LogSoftmax for multi-class classification
    )

optimizer_dict = {'Momentum' : optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001),
                'NAG': optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True,weight_decay=0.0001),
                'Adam': optim.Adam(model.parameters(), lr=learning_rate,weight_decay=0.0001),
                'Adagrad': optim.Adagrad(model.parameters(), lr=learning_rate, lr_decay=0, weight_decay=0.0001, initial_accumulator_value=0, eps=1e-10),
                 'SGD': optim.SGD(model.parameters(), lr=learning_rate,weight_decay=0.0001)
                }
lr_list = [ 0.01, 0.001, 0.0001]
batch_size_list = [56,128,256,512,1024]
num_epochs = 10000 

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
Res=pd.DataFrame(columns=['model','optimizer','lr','batch_size','early_ending', 'final_score_train','final_score','macro_f1','macro_tpr_gap'])
i=0
for opt_name, optimizer in optimizer_dict.items():
    for learning_rate in lr_list:
        for batch_size in batch_size_list:
            name = 'NN-28-28_'+opt_name+'_lr_'+str(learning_rate)+'_batch_size_'+str(batch_size)+str(i)
            print('\n\nStarting to train model', name)
            model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap , early_ending, final_score_train= train_NN_with_custom_loss(model, optimizer, batch_size, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
            Res.loc[i]=[name,opt_name,learning_rate,batch_size,early_ending,final_score_train,final_score, macro_f1, inv_macro_tpr_gap]
            save_Y_pred_tofile(X_test_true_tensor, model_trained,name)
            i+=1




Starting to train model NN-28-28_Momentum_lr_0.01_batch_size_560
Epoch 1, Loss: 0.9643118381500244, Final Score Train: 0.5033621788024902, Final Score Test: 0.503541886806488 (gap 0.00017970800399780273) macro F1 Train: 0.006724300754334216, macro F1 Test: 0.007083729450228395, 1-TPR Gap Train: 1.0, 1-TPR Gap Test: 1.0
Arrêt précoce après 6 époques
Final Evaluation Score: 0.503541886806488 gap 0.00017970800399780273 || Macro F1: 0.0162663060769965 1-TPR_gap: 1.0


Starting to train model NN-28-28_Momentum_lr_0.01_batch_size_1281
Epoch 1, Loss: 0.9643909931182861, Final Score Train: 0.5033621788024902, Final Score Test: 0.503541886806488 (gap 0.00017970800399780273) macro F1 Train: 0.006724300754334216, macro F1 Test: 0.007083729450228395, 1-TPR Gap Train: 1.0, 1-TPR Gap Test: 1.0
Epoch 10, Loss: 0.9644181728363037, Final Score Train: 0.5033621788024902, Final Score Test: 0.503541886806488 (gap 0.00017970800399780273) macro F1 Train: 0.006724300754334216, macro F1 Test: 0.007083729450

In [ ]:
path_pkl = ''

with open('RESULTS_NN-28-28_12-03-2024_decay.pkl', 'wb') as f:
   pickle.dump(Res, f)

path_pkl = 'pkl_files/'
Res = pd.read_pickle('RESULTS_NN-28-28_12-03-2024_decay.pkl')
   
Res.head()

,model,optimizer,lr,batch_size,early_ending,final_score_train,final_score,macro_f1,macro_tpr_gap


In [ ]:
# 7 leanring rate x 7 batch size = 49 combinaisons par optimizer
# 5 optimizer x 49 combinaison = 245
Res.iloc[97:144,:].sort_values(by='batch_size').head(49)  #'batch_size'

,model,optimizer,lr,batch_size,early_ending,final_score_train,final_score,macro_f1,macro_tpr_gap


In [ ]:
#print(Res[Res['optimizer']==list(optimizer_dict.keys())[2]])
list(optimizer_dict.keys())[0]

'Momentum'

In [ ]:
#################################################
#          BOUCLE HYPERPARAMETRES
################################################


# 1. Define the model and optimizer and train
# --------------------------------------------------

model_2 = nn.Sequential(
    nn.Linear(768, 2048),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Dropout(p=0.5),
    nn.Linear(2048, 256),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Dropout(p=0.3),
    nn.Linear(256, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1)  # LogSoftmax for multi-class classification
    )

optimizer_dict = {'Momentum' : optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001),
                'NAG': optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True,weight_decay=0.0001),
                'Adam': optim.Adam(model.parameters(), lr=learning_rate,weight_decay=0.0001),
                'Adagrad': optim.Adagrad(model.parameters(), lr=learning_rate, lr_decay=0, weight_decay=0.0001, initial_accumulator_value=0, eps=1e-10),
                 'SGD': optim.SGD(model.parameters(), lr=learning_rate,weight_decay=0.0001)
                }
lr_list = [ 0.01, 0.001, 0.0001]
batch_size_list = [56,128,256,512,1024]
num_epochs = 10000 

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
Res_2=pd.DataFrame(columns=['model','optimizer','lr','batch_size','early_ending', 'final_score','gap','final_score','macro_f1','macro_tpr_gap'])
i=0
for opt_name, optimizer in optimizer_dict.items():
    for learning_rate in lr_list:
        for batch_size in batch_size_list:
            name = 'NN-2048-256-28_'+opt_name+'_lr_'+str(learning_rate)+'_batch_size_'+str(batch_size)+'_'+str(i)
            print('\n\nStarting to train model', name)
            model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap , early_ending , final_score_train = train_NN_with_custom_loss(model_2, optimizer, batch_size, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
            Res_2.loc[i]=[name,opt_name,learning_rate,batch_size,early_ending,final_score_train, final_score_train - final_score, final_score, macro_f1, inv_macro_tpr_gap]
            save_Y_pred_tofile(X_test_true_tensor, model_trained,name)
            i+=1




Starting to train model NN-2048-256-28_Momentum_lr_0.01_batch_size_56_0
Epoch 1, Loss: 0.9622682332992554, Final Score Train: 0.49911874532699585, Final Score Test: 0.49868887662887573 (gap -0.0004298686981201172) macro F1 Train: 0.00910291207674675, macro F1 Test: 0.008741462217204633, 1-TPR Gap Train: 0.9891345500946045, 1-TPR Gap Test: 0.988636314868927
Epoch 10, Loss: 0.9625880718231201, Final Score Train: 0.49911874532699585, Final Score Test: 0.49868887662887573 (gap -0.0004298686981201172) macro F1 Train: 0.00910291207674675, macro F1 Test: 0.008741462217204633, 1-TPR Gap Train: 0.9891345500946045, 1-TPR Gap Test: 0.988636314868927
Arrêt précoce après 11 époques
Final Evaluation Score: 0.49868887662887573 gap -0.0004298686981201172 || Macro F1: 0.008741462217204633 1-TPR_gap: 0.988636314868927


Starting to train model NN-2048-256-28_NAG_lr_0.01_batch_size_56_1
Epoch 1, Loss: 0.9614269733428955, Final Score Train: 0.49911874532699585, Final Score Test: 0.49868887662887573 (gap

In [ ]:
path_pkl = ''

with open(path_pkl + 'RESULTS_NN-2048-256-28_12-03-2024.pkl', 'wb') as f:
   pickle.dump(Res_2, f)

#path_pkl = 'pkl_files/'
#train = pd.read_pickle(path_pkl + 'train_pp.pkl')
   
   Res_2

In [11]:
Res_2[Res_2['optimizer']=='Adam']

,model,optimizer,lr,batch_size,early_ending,final_score,gap,final_score,macro_f1,macro_tpr_gap
2,NN-2048-256-28_Adam_lr_0.01_batch_size_56_2,Adam,0.01,56,11,tensor(0.4991),tensor(0.0004),tensor(0.4987),0.008741,tensor(0.9886)


In [ ]:
#ENTRAINEMENT SUR TOUT X

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
optimizer_dict = {'Momentum' : optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001),
                'NAG': optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True,weight_decay=0.0001),
                'Adam': optim.Adam(model.parameters(), lr=learning_rate,weight_decay=0.0001),
                'Adagrad': optim.Adagrad(model.parameters(), lr=learning_rate, lr_decay=0, weight_decay=0.0001, initial_accumulator_value=0, eps=1e-10),
                 'SGD': optim.SGD(model.parameters(), lr=learning_rate,weight_decay=0.0001)
                }
lr_list = [ 0.01, 0.001, 0.0001]
batch_size_list = [56,128,256,512,1024]
num_epochs = 10000 

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
Res_2_all=pd.DataFrame(columns=['model','optimizer','lr','batch_size','early_ending', 'final_score','gap','final_score','macro_f1','macro_tpr_gap'])

i=0
for opt_name, optimizer in optimizer_dict.items():
    for learning_rate in lr_list[:1]:
        for batch_size in batch_size_list[:1]:
            name = 'NN-2048-256-28_'+opt_name+'_lr_'+str(learning_rate)+'_batch_size_'+str(batch_size)+'_'+str(i)
            print('\n\nStarting to train model', name)
            model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap , early_ending , final_score_train = train_NN_with_custom_loss(model_2, optimizer, batch_size, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
            Res_2.loc[i]=[name,opt_name,learning_rate,batch_size,early_ending,final_score_train, final_score_train - final_score, final_score, macro_f1, inv_macro_tpr_gap]
            save_Y_pred_tofile(X_test_true_tensor, model_trained,name)
            i+=1

In [ ]:
model = nn.Sequential(
    nn.Linear(768, 2048),  # Couche d'entrée à la première couche cachée
    nn.ReLU(),  # Fonction d'activation ReLU
    nn.Dropout(p=0.5),  # Dropout avec une probabilité de désactivation de 50%
    nn.Linear(2048, 512),  # De la première couche cachée à la deuxième couche cachée
    nn.ReLU(),  # Une autre fonction d'activation ReLU après la deuxième couche cachée
    nn.Dropout(p=0.5),  # Un autre dropout après la deuxième couche cachée
    nn.Linear(512, 28),  # De la deuxième couche cachée à la couche de sortie
    nn.LogSoftmax(dim=1)  # LogSoftmax pour la classification multiclasse
)

learning_rate = 0.01
num_epochs = 20000 

name = 'NN2048-512-28-dropout_Adam'+'_lr_'+str(learning_rate)+'_alpha_5'
print('\n\n Starting to train model', name)
model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap = train_NN_with_custom_loss(model,optim.Adam(model.parameters(), lr=learning_rate) , 5, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
Res.loc[i]=[name,optimizer,learning_rate,alpha,final_score, macro_f1, inv_macro_tpr_gap]
save_Y_pred_tofile(X_test_true_tensor, model_trained,name)

In [ ]:
model = nn.Sequential(
    nn.Linear(768, 2048),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(2048,512),  # Assuming 768 input features and 28 classes
    nn.Linear(512,28),  
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1))  # LogSoftmax for multi-class classification

learning_rate = 0.01
num_epochs = 20000 

name = 'NN2048-512-28_Adam'+'_lr_'+str(learning_rate)+'_alpha_5'
print('\n\n Starting to train model', name)
model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap = train_NN_with_custom_loss(model,optim.Adam(model.parameters(), lr=learning_rate) , 5, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)
Res.loc[i]=[name,optimizer,learning_rate,alpha,final_score, macro_f1, inv_macro_tpr_gap]
save_Y_pred_tofile(X_test_true_tensor, model_trained,name)

In [ ]:
path_pkl = ''

with open(path_pkl + 'RESULTS_11-03-2024.pkl', 'wb') as f:
   pickle.dump(Res, f)

#path_pkl = 'pkl_files/'
#train = pd.read_pickle(path_pkl + 'train_pp.pkl')

In [ ]:
# Make predictions
with torch.no_grad():  # We do not need gradient computation for prediction
    model.eval()  # Set the model to evaluation mode
    Y_pred_probs = model(X_test_tensor)
    Y_pred = torch.argmax(Y_pred_probs, dim=1)  # Get the class with the highest probability

# Convert Y_pred to a DataFrame
Y_pred_df = pd.DataFrame(Y_pred.numpy(), columns=['Predicted'])

# Evaluate Y_pred compared to Y_test (assuming Y_test is a numpy array or a pandas Series)
print(classification_report(Y_test, Y_pred_df['Predicted']))


**OPTIMIZING TRAINING FUNCTION**
---

In [ ]:
# AVEC MINI BATCH

def train_NN_with_custom_loss(model, optimizer, batch_size, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor):

    # 1. Convertir les tensors en datasets puis en DataLoader pour gérer les mini-batchs
    train_dataset = TensorDataset(X_train_tensor, Y_train_one_hot, S_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = TensorDataset(X_test_tensor, Y_test_one_hot, S_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    
    # 2. Paramètres pour l'arrêt précoce
    # -------------------------------
    patience = 5  # Nombre d'époques à attendre après la dernière amélioration de la perte de validation
    best_loss = None
    early_ending = None
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        
        # 1/ exécuter les minibatches et recupérer la loss moyenne
        for X_batch, Y_batch, S_batch in train_loader:
            # Y_batch est one hot
            
            model.train()
            optimizer.zero_grad()
            outputs_train = model(X_batch)
            loss = soft_final_score_loss(Y_batch, outputs_train, S_batch)
            loss.backward()
            optimizer.step()

            # save mini-batch loss
            train_loss += loss.item()
        
        # Average loss pour l'epoch (après boucle mini-batchs)
        train_loss = train_loss / len(train_loader)       
        
        # 2. Vérifier si la perte de validation s'est améliorée (arret précoce)

        # Evaluation sur le jeu de données de test
        model.eval()
        test_loss = 0.0
        
        with torch.no_grad():
            for X_batch_test, Y_batch_test, S_batch_test in test_loader:
                outputs_test = model(X_batch_test)
                #Y_batch_test_one_hot = torch.nn.functional.one_hot(Y_batch_test, num_classes=Y_train.nunique())
                loss_test = soft_final_score_loss(Y_batch_test, outputs_test, S_batch_test)
                test_loss += loss_test.item()
                
        #average_test_loss = running_loss_test / len(test_loader)
        test_loss = test_loss / len(test_loader)
       
        # check if improvement in loss (compared to last epoch)
        if best_loss is None or test_loss < best_loss:
            best_loss = test_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f'Arrêt précoce après {epoch+1} époques')
                early_ending = epoch + 1
                break  # Arrêter l'entraînement
        
        # 3. Impression de l'apprentissage et des scores train et test
        if epoch==0 or (epoch+1) % 10 == 0:
            
            with torch.no_grad():
                
                # Calculate metrics for training data
                outputs_train = model(X_train_tensor) # probabilities
                # Evaluate predictions on training data
                final_score_train_ = get_final_score(Y_train_tensor, outputs_train, S_train_tensor)
                macro_f1_train = get_macro_f1(Y_train_tensor, outputs_train)
                inv_macro_tpr_gap_train = 1 - get_macro_tpr_gap(Y_train_tensor, outputs_train, S_train_tensor)
            
                # Calculate metrics for test data
                outputs_test = model(X_test_tensor)
                # Evaluate predictions on training data
                final_score_test_ = get_final_score(Y_test_tensor, outputs_test, S_test_tensor)
                macro_f1_test = get_macro_f1(Y_test_tensor, outputs_test)
                inv_macro_tpr_gap_test = 1 - get_macro_tpr_gap(Y_test_tensor, outputs_test, S_test_tensor)

                print(f'Epoch {epoch+1}, Loss: {loss.item()}, Final Score Train: {final_score_train_.item()}, Final Score Test: {final_score_test_.item()} (gap {final_score_test_-final_score_train_}) macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')
            
    # 4. Make Predictions and Evaluate with final_score
    # -------------------------------------------------
            
    with torch.no_grad():
        model.eval()

        Y_train_pred_probs = model(X_train_tensor) # dim = 28 (Probabilities for each class)
        
        # # Y_train_pred_tensor = torch.argmax(Y_train_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
        final_score_train = get_final_score(Y_train_tensor, Y_train_pred_probs, S_train_tensor)

        Y_pred_probs = model(X_test_tensor) # dim = 28 (Probabilities for each class)
        # Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
        macro_f1 = get_macro_f1(Y_test_tensor, Y_pred_tensor)
        inv_macro_tpr_gap = 1 - get_macro_tpr_gap(Y_test_tensor, Y_pred_probs, S_test_tensor)
        final_score = get_final_score(Y_test_tensor, Y_pred_probs, S_test_tensor)
        
        print(f'Final Evaluation Score: {final_score.item()} gap {final_score.item()-final_score_train.item()} || Macro F1: {macro_f1.item()} 1-TPR_gap: { inv_macro_tpr_gap.item() }')

    return model, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap, early_ending,final_score_train